Make sure to use this against standard cluster but not Serverless compute 

Serverless compute can not access the dbfs as this is a shared compute!

In [0]:
%sh
rm -r /dbfs/delta_lab
mkdir /dbfs/delta_lab
wget -O /dbfs/delta_lab/products.csv https://raw.githubusercontent.com/kuljotSB/DatabricksUdemyCourse/refs/heads/main/DataAnalytics/products.csv

--2025-07-14 11:08:23--  https://raw.githubusercontent.com/kuljotSB/DatabricksUdemyCourse/refs/heads/main/DataAnalytics/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14372 (14K) [text/plain]
Saving to: ‘/dbfs/delta_lab/products.csv’

     0K .......... ....                                       100% 2.70M=0.005s

2025-07-14 11:08:24 (2.70 MB/s) - ‘/dbfs/delta_lab/products.csv’ saved [14372/14372]



In [0]:
%sh
ls /dbfs

Volume
Volumes
databricks
databricks-datasets
databricks-results
delta_lab
mnt
tmp
volume
volumes


In [0]:
%sql 

CREATE OR REFRESH LIVE TABLE raw_products_data
COMMENT "Products sample dataset"
AS
SELECT
 ProductID,
 ProductName,
 Category,
 ListPrice
FROM read_files('dbfs:/delta_lab/products.csv', format => 'csv', header => true)

Name,Type
ProductID,int
ProductName,string
Category,string
ListPrice,double


In [0]:
%sql
CREATE OR REFRESH LIVE TABLE aggregated_products_data
COMMENT "Aggregated data"
AS
SELECT
   Category,
   sum(ListPrice) as Total_Price
FROM live.raw_products_data
GROUP BY Category;

Name,Type
Category,string
Total_Price,double


In [0]:
df = spark.read.format("delta").load('/pipelines/delta_lab/tables/aggregated_products_data')
display(df)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7907898415166376>, line 1
----> 1 df = spark.read.format("delta").load('/pipelines/delta_lab/tables/aggregated_products_data')
      2 display(df)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
    313 elif path is not None:
    314     if type(path) != list:

Fi